This notebook is for testing/developing the sampling of `_Population_Discrete` models, typically the `Discrete_Illustris` population model.  (Re)Sampling of the population is performed using `kalepy`.
* **2022-08-17 - BUG**: Problem with current sampling implementation.  The number of binary samples depends on the number/spacing of input frequencies, while it should be independent of this.

In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

## Calculate GWB Without fully sampling the Universe

In [ ]:
resamp = holo.population.PM_Resample(1.0)
pop = holo.population.Pop_Illustris(mods=resamp)
print(f"{pop.size=}")

fixed = holo.evolution.Fixed_Time.from_pop(pop, 2.0 * GYR)
evo = holo.evolution.Evolution(pop, fixed)
evo.evolve()

In [ ]:
def _calc(fobs_gw_edges, evo, down_sample):
    """
    
    fobs_gw_edges : GW-frequencies observer-frame
    
    """
    fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges, log=False)
    dlnf = np.diff(np.log(fobs_gw_edges))

    PARAMS = ['mass', 'sepa', 'dadt', 'scafa']
    # convert from GW to orbital frequencies
    fobs_orb_edges = fobs_gw_edges / 2.0
    fobs_orb_cents = fobs_gw_cents / 2.0
    data_fobs = evo.at('fobs', fobs_orb_cents, params=PARAMS)
    
    redz = cosmo.a_to_z(data_fobs['scafa'])
    valid = (redz > 0.0)
    # rest-frame GW-frequencies
    frst_gw_cents = utils.frst_from_fobs(fobs_gw_cents[np.newaxis, :], redz)
    frst_orb_cents = frst_gw_cents / 2.0
    dcom = cosmo.z_to_dcom(redz)
    m1, m2 = np.moveaxis(data_fobs['mass'], -1, 0)
    # convert from GW- to orbital-fequencies
    dfdt, _ = utils.dfdt_from_dadt(data_fobs['dadt'], data_fobs['sepa'], frst_orb=frst_orb_cents)

    _lambda_factor = utils.lambda_factor_freq(frst_orb_cents, dfdt, redz, dcom=dcom) / evo._sample_volume
    lambda_factor = _lambda_factor * dlnf[np.newaxis, :]

    # ! ======
    mchirp = utils.chirp_mass(m1, m2)
    hs2 = utils.gw_strain_source(mchirp, dcom, frst_orb_cents)**2
    gwb_direct = hs2 * np.random.poisson(np.nan_to_num(_lambda_factor))
    gwb_direct[~valid] = 0.0
    gwb_direct = np.sum(gwb_direct, axis=0)
    gwb_direct = np.sqrt(gwb_direct)
    # ! ======

    # select only valid entries
    mt, mr = utils.mtmr_from_m1m2(m1[valid], m2[valid])
    # broadcast `fobs` to match the shape of binaries, then select valid entries
    fo = (fobs_orb_cents[np.newaxis, :] * np.ones_like(redz))[valid]
    redz = redz[valid]
    weights = lambda_factor[valid]

    # down-sample weights to decrease the number of sample points
    weights /= down_sample

    # Convert to log-space
    vals = [np.log10(mt), np.log10(mr), np.log10(redz), np.log10(fo)]
    names = ['mtot', 'mrat', 'redz', 'fobs']
    nsamp = np.random.poisson(weights.sum())
    reflect = [None, [None, 0.0], None, np.log10([fobs_orb_edges[0], fobs_orb_edges[-1]])]
    samples = kale.resample(vals, size=nsamp, reflect=reflect, weights=np.random.poisson(weights), bw_rescale=0.5)

    # Convert back to normal-space
    samples = np.asarray([10.0 ** ss for ss in samples])
    vals = np.asarray([10.0 ** vv for vv in vals])
    
    samples_weights = np.ones_like(samples[0]) * down_sample
        
    weights *= down_sample
    weights = np.random.poisson(weights)
    *_, gwb_samples = holo.sam._gws_from_samples(samples, samples_weights, fobs_gw_edges)
    *_, gwb_vals = holo.sam._gws_from_samples(vals, weights, fobs_gw_edges)
    
    gwdis = holo.gravwaves.GW_Discrete(evo, fobs_gw_cents, nreals=1)
    gwdis.emit(progress=False)
    gwb_discrete = gwdis.both.squeeze()
    
    return gwb_discrete, gwb_direct, gwb_samples, gwb_vals


def calc(fobs_gw_edges, evo, down_sample, nreals):
    shape = (fobs_gw_edges.size - 1, nreals)
    gwb_discrete = np.zeros(shape)
    gwb_direct = np.zeros(shape)
    gwb_samples = np.zeros(shape)
    gwb_vals = np.zeros(shape)
    for rr in tqdm.trange(nreals):
        gwb_discrete[:, rr], gwb_direct[:, rr], gwb_samples[:, rr], gwb_vals[:, rr] = \
            _calc(fobs_gw_edges, evo, down_sample)

    return gwb_discrete, gwb_direct, gwb_samples, gwb_vals

freqs = holo.utils.nyquist_freqs(dur=10*YR, cad=0.01*YR)
gwb_discrete, gwb_direct, gwb_samples, gwb_vals = calc(freqs, evo, down_sample=1e1, nreals=10)

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = kale.utils.midpoints(freqs) * YR
plot._draw_plaw(ax, xx, 1e-15, 1)

# ---- Discrete
gwb_list = [gwb_discrete, gwb_direct, gwb_samples, gwb_vals]
gwb_names = ['gwb_discrete', 'gwb_direct', 'gwb_samples', 'gwb_vals']
for gwb, lab in zip(gwb_list, gwb_names):
    med, *conf = np.percentile(gwb, [50, 25, 75], axis=-1)
    hh, = ax.plot(xx, med, label=lab, lw=1.5)
    col = hh.get_color()
    ax.fill_between(xx, *conf, alpha=0.1, color=col)
    size = gwb.shape[1]
    sel = np.min([size, 5])
    sel = np.random.choice(size, sel, replace=False)
    ax.plot(xx, gwb[:, sel], color=col, alpha=0.35, lw=0.5)

ax.legend()
plt.show()

In [ ]:
def _calc(fobs_gw_edges, evo, down_sample):
    """
    
    fobs_gw_edges : GW-frequencies observer-frame
    
    """
    fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges, log=False)
    dlnf = np.diff(np.log(fobs_gw_edges))

    PARAMS = ['mass', 'sepa', 'dadt', 'scafa']
    # convert from GW to orbital frequencies
    # fobs_orb_edges = fobs_gw_edges / 2.0
    fobs_orb_cents = fobs_gw_cents / 2.0
    data_fobs = evo.at('fobs', fobs_orb_cents, params=PARAMS)
    
    redz = cosmo.a_to_z(data_fobs['scafa'])
    valid = (redz > 0.0)
    # rest-frame GW-frequencies
    frst_gw_cents = utils.frst_from_fobs(fobs_gw_cents[np.newaxis, :], redz)
    frst_orb_cents = frst_gw_cents / 2.0
    dcom = cosmo.z_to_dcom(redz)
    m1, m2 = np.moveaxis(data_fobs['mass'], -1, 0)
    # convert from GW- to orbital-fequencies
    dfdt, _ = utils.dfdt_from_dadt(data_fobs['dadt'], data_fobs['sepa'], frst_orb=frst_orb_cents)

    _lambda_factor = utils.lambda_factor_freq(frst_orb_cents, dfdt, redz, dcom=dcom) / evo._sample_volume
    lambda_factor = _lambda_factor * dlnf[np.newaxis, :]

    # ! ---- Direct ----
    mchirp = utils.chirp_mass(m1, m2)
    hs2 = utils.gw_strain_source(mchirp, dcom, frst_orb_cents)**2
    gwb_direct = np.zeros_like(hs2)
    gwb_direct[valid] = hs2[valid] * np.random.poisson(_lambda_factor[valid])   # should be this?
    # gwb_direct[valid] = hs2[valid] * np.random.poisson(lambda_factor[valid])
    gwb_direct = np.sum(gwb_direct, axis=0)
    gwb_direct = np.sqrt(gwb_direct)


    # ! ---- Sample Values ----
    # select only valid entries
    mt, mr = utils.mtmr_from_m1m2(m1[valid], m2[valid])
    # broadcast `fobs` to match the shape of binaries, then select valid entries
    fo = (fobs_orb_cents[np.newaxis, :] * np.ones_like(redz))[valid]
    redz = redz[valid]
    weights = lambda_factor[valid]

    # down-sample weights to decrease the number of sample points
    weights /= down_sample

    # Convert to log-space
    # vals = [np.log10(mt), np.log10(mr), np.log10(redz), np.log10(fo)]
    # vals = np.asarray([10.0 ** vv for vv in vals])
    
    vals = np.asarray([mt, mr, redz, fo])
    weights *= down_sample
    weights = np.random.poisson(weights)
    *_, gwb_vals = holo.sam._gws_from_samples(vals, weights, fobs_gw_edges)
    
    return gwb_direct, gwb_vals


def calc(fobs_gw_edges, evo, down_sample, nreals):
    shape = (fobs_gw_edges.size - 1, nreals)
    gwb_direct = np.zeros(shape)
    gwb_vals = np.zeros(shape)
    for rr in tqdm.trange(nreals):
        gwb_direct[:, rr], gwb_vals[:, rr] = \
            _calc(fobs_gw_edges, evo, down_sample)

    return gwb_direct, gwb_vals

# freqs = holo.utils.nyquist_freqs(dur=10*YR, cad=0.1*YR)
freqs = holo.utils.nyquist_freqs(dur=1*YR, cad=0.02*YR)
gwb_direct, gwb_vals = calc(freqs, evo, down_sample=1e1, nreals=30)

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = kale.utils.midpoints(freqs) * YR
plot._draw_plaw(ax, xx, 1e-15, 1)

# ---- Discrete
gwb_list = [gwb_direct, gwb_vals]
gwb_names = ['gwb_direct', 'gwb_vals']
for gwb, lab in zip(gwb_list, gwb_names):
    med, *conf = np.percentile(gwb, [50, 25, 75], axis=-1)
    hh, = ax.plot(xx, med, label=lab, lw=1.5)
    col = hh.get_color()
    ax.fill_between(xx, *conf, alpha=0.1, color=col)
    size = gwb.shape[1]
    sel = np.min([size, 5])
    sel = np.random.choice(size, sel, replace=False)
    ax.plot(xx, gwb[:, sel], color=col, alpha=0.35, lw=0.5)

ax.legend()
plt.show()

In [ ]:
# construct sampling frequencies
freqs_volumetric = holo.utils.nyquist_freqs(dur=10.0*YR, cad=0.1*YR)
# calculate discretized GW signals
gwb_volumetric = holo.gravwaves.GW_Discrete(evo, freqs_volumetric, nreals=100)
gwb_volumetric.emit()

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')

xx = freqs_volumetric * YR

med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med)
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

plot._twin_hz(ax)
plt.show()

## Fully sample Universe

In [ ]:
NUM = 10
freqs_samples = np.logspace(0, 1, NUM) / YR

names, samples_10 = evo._sample_universe(freqs_samples)
num_samp_10 = samples_10[0].size
print(names, samples_10[0].shape, f"{num_samp_10:.4e}")

In [ ]:
print(names, samples_10[0].shape, f"{num_samp_10:.4e}")

In [ ]:
fobs = holo.utils.nyquist_freqs(dur=1.0*YR, cad=0.1*YR)

In [ ]:
weights_10 = np.ones_like(samples_10[0])
gff_10, gwf_10, gwb_10 = holo.sam._gws_from_samples(samples_10, weights_10, fobs)

In [ ]:
NUM = 100
freqs_samples = np.logspace(0, 1, NUM) / YR

names, samples_100 = evo._sample_universe(freqs_samples)
num_samp_100 = samples_100[0].size
print(names, samples_100[0].shape, f"{num_samp_100:.4e}")

In [ ]:
print(names, samples_100[0].shape, f"{num_samp_100:.4e}")

In [ ]:
weights_100 = np.ones_like(samples_100[0])
gff_100, gwf_100, gwb_100 = holo.sam._gws_from_samples(samples_100, weights_100, fobs)

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = freqs_volumetric * YR
med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med, 'k--')
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

xx = gff_10 * YR
hh = ax.scatter(xx, gwf_10)
plot.draw_hist_steps(ax, fobs*YR, gwb_10, color=hh.get_facecolor())

xx = gff_100 * YR
hh = ax.scatter(xx, gwf_100)
plot.draw_hist_steps(ax, fobs*YR, gwb_100, color=hh.get_facecolor())


plt.show()

## Test

In [ ]:
# NUM = 10
DOWN = None
# DOWN = 1e1
# freqs = np.logspace(0, 1, NUM) / YR
fobs = holo.utils.nyquist_freqs(dur=1.0*YR, cad=0.1*YR)
freqs = fobs

REALS = 1
gff = np.zeros((fobs.size - 1, REALS))
gwf = np.zeros_like(gff)
gwb = np.zeros_like(gff)
check_1 = np.zeros_like(gff)
check_2 = np.zeros_like(gff)
for rr in range(REALS):
    print("\n", rr)
    names, samples, check_1[:, rr], temp, temp_ww = evo._sample_universe(freqs, down_sample=DOWN)
    weights = np.ones_like(samples[0])
    if DOWN is not None:
        weights *= DOWN
    gff[:, rr], gwf[:, rr], gwb[:, rr] = holo.sam._gws_from_samples(samples, weights, fobs)
    _gff, _gwf, check_2[:, rr] = holo.sam._gws_from_samples(temp, temp_ww, fobs)

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)
plot._draw_plaw(ax, [0.1, 10], 1e-15, 1)

xx = freqs_volumetric * YR
med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med, 'k--')
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

xx = fobs * YR
yy = np.median(check_1, axis=-1)
# yy = np.mean(check, axis=-1)
plot.draw_hist_steps(ax, xx, yy, color='r', ls='--')
print(yy)

yy = np.median(check_2, axis=-1)
# yy = np.mean(check, axis=-1)
plot.draw_hist_steps(ax, xx, yy, color='r', ls=':')

med, *conf = np.percentile(gwb, [50, 25, 75], axis=-1)
hh, = plot.draw_hist_steps(ax, xx, med)
_xx, lo = plot._get_hist_steps(xx, conf[0])
_xx, hi = plot._get_hist_steps(xx, conf[1])
ax.fill_between(_xx, lo, hi, alpha=0.2, color=hh.get_color())
# xx = gff * YR
# hh = ax.scatter(xx, gwf)

plt.show()

In [ ]:
NUM = 10
DOWN = None
freqs = np.logspace(0, 1, NUM) / YR
fobs = holo.utils.nyquist_freqs(dur=1.0*YR, cad=0.1*YR)

names, samples = evo._sample_universe(freqs, down_sample=DOWN)
num_samp = samples[0].size
print(names, samples[0].shape, f"{num_samp:.4e}")

weights = np.ones_like(samples[0])
if DOWN is not None:
    weights *= DOWN
gff, gwf, gwb = holo.sam._gws_from_samples(samples, weights, fobs)


fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = freqs_volumetric * YR
med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med, 'k--')
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

xx = gff * YR
hh = ax.scatter(xx, gwf)
plot.draw_hist_steps(ax, fobs*YR, gwb, color=hh.get_facecolor())

plt.show()

In [ ]:
NUM = 100
DOWN = None
freqs = np.logspace(0, 1, NUM) / YR
fobs = holo.utils.nyquist_freqs(dur=1.0*YR, cad=0.1*YR)

names, samples = evo._sample_universe(freqs, down_sample=DOWN)
num_samp = samples[0].size
print(names, samples[0].shape, f"{num_samp:.4e}")

weights = np.ones_like(samples[0])
if DOWN is not None:
    weights *= DOWN
gff, gwf, gwb = holo.sam._gws_from_samples(samples, weights, fobs)


fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = freqs_volumetric * YR
med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med, 'k--')
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

xx = gff * YR
hh = ax.scatter(xx, gwf)
plot.draw_hist_steps(ax, fobs*YR, gwb, color=hh.get_facecolor())

plt.show()

In [ ]:
NUM = 100
DOWN = None
freqs = np.logspace(0, 1, NUM) / YR
fobs = holo.utils.nyquist_freqs(dur=1.0*YR, cad=0.1*YR)

names, samples = evo._sample_universe(freqs, down_sample=DOWN)
num_samp = samples[0].size
print(names, samples[0].shape, f"{num_samp:.4e}")

weights = np.ones_like(samples[0])
if DOWN is not None:
    weights *= DOWN
gff, gwf, gwb = holo.sam._gws_from_samples(samples, weights, fobs)


fig, ax = plot.figax(xlabel='Frequency $[\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

xx = freqs_volumetric * YR
med, *conf = np.percentile(gwb_volumetric.back, [50, 25, 75], axis=-1)
hh, = ax.plot(xx, med, 'k--')
ax.fill_between(xx, *conf, alpha=0.1, color=hh.get_color())

xx = gff * YR
hh = ax.scatter(xx, gwf)
plot.draw_hist_steps(ax, freqs*YR, gwb, color=hh.get_facecolor())

plt.show()

## Other

There should be a better way to sample in frequencies because we know the power-law index of the hardening rate (and thus the expected number) of sources vs. frequencies.
* Do normal `kalepy` resampling, and then override the frequencies manually?
* Do grid-resampling in frequency, and KDE resampling for other parameters?